In [1]:
import json
import re
import nltk
from nltk.corpus import wordnet
import calendar
import gender_guesser.detector as gender
import random

## Band Name Placeholder

In [2]:
def band_placeholder(data, text):
    band_name = data['Name']
    if '(' in band_name: # some bands have disambiguation in the title such as "Mother Earth (American band)"
        band_name = ' '.join(band_name.split(' (')[:-1])
    text = re.sub(band_name, '[BAND_NAME]', text)
    return text

## Year Placeholder

In [3]:
def year_placeholder(text):
    return re.sub("[0-9]{4}", '[YEAR]', text)

## Month Placeholder

In [4]:
def month_placeholder(text):
    months = [calendar.month_name[i] for i in range(1,13)] + [calendar.month_abbr[i] for i in range(1,13)]
    for month in months:
        text = re.sub(month, '[MONTH]', text)
        
    return text

## Name Placeholder
https://stackoverflow.com/questions/20290870/improving-the-extraction-of-human-names-with-nltk
Shivansh bhandari's answer

In [5]:
def get_person_names(data, text):
    person_list = []
    person_names=person_list
    def get_human_names(text):
        tokens = nltk.tokenize.word_tokenize(text)
        pos = nltk.pos_tag(tokens)
        sentt = nltk.ne_chunk(pos, binary = False)

        person = []
        name = ""
        for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
            for leaf in subtree.leaves():
                person.append(leaf[0])
            if len(person) > 1: #avoid grabbing lone surnames
                for part in person:
                    name += part + ' '
                if name[:-1] not in person_list:
                    person_list.append(name[:-1])
                name = ''
            person = []

    names_ = get_human_names(text)
    for person in person_list:
        person_split = person.split(" ")
        for name in person_split:
            if wordnet.synsets(name):
                if(name in person):
                    person_names.remove(person)
                    break
                    
    # add member names from infobox
    if 'infobox' in data.keys():
        members = []
        if 'Members' in data['infobox'].keys():
            members += data['infobox']['Members']

        if 'Past members' in data['infobox'].keys():
            members += data['infobox']['Past members']
            
        # some members have extra information that needs to be removed such as 'Earl Yager - bass'
        for member in members:
            member = member.split()
            if len(member) > 1:
                member = member[0] + member[1]
            else:
                member = member[0]

            person_names.append(member)
            
    person_names = set(person_names)
    return person_names

In [6]:
def get_name_genders(person_names):
    name_genders = {}
    d = gender.Detector()
    for name in person_names:
        g = d.get_gender(name.split()[0])
        if 'female' in g:
            name_genders[name] = 'FEMALE'
        elif 'male' in g:
            name_genders[name] = 'MALE'
        elif g == 'andy':
            name_genders[name] = random.choice(['MALE', 'FEMALE'])
        else:
            continue # unknown name
            
    return name_genders

In [7]:
def person_name_placeholder(data, text):
    person_names = get_person_names(data, text)
    name_genders = get_name_genders(person_names)
    i = 0
    for name,gender in name_genders.items():
        text = re.sub(name, '[PERSON_NAME_FULL_' + str(i) + '_' + gender + ']', text)
        if len(name.split()) > 1: # some only have first names
            text = re.sub(name.split()[0], '[PERSON_NAME_FIRST_' + str(i) + ']', text) # replace first name
            text = re.sub(name.split()[1], '[PERSON_NAME_LAST_' + str(i) + ']', text) # replace last name
        i += 1
        
    return text

## Genre Placeholder

### Get all genres

In [8]:
with open('data/consolidatedData.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
genres = set()
for genre in data['allGenres']:
    if isinstance(genre, str):
        g = re.split(',|\[', genre)[0]
        genres.add(g)
        
genres = list(genres) # need to convert to list to be able to take a random choice from it

In [16]:
def genre_placeholder(text):
    part2 = re.split(' a | an ', text)
    genre = None
    if len(part2) > 1:
        genre = re.split(' band', part2[1])[0]
    print(part2)   
    print(genre)
    if genre:
        text = re.sub(genre, '[GENRE]', text)
    
    return text

## Paragraph Placeholder

In [10]:
def get_paragraph_placeholder(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    text = data['rawData']
    text = ''.join(text)
    text = band_placeholder(data, text)
    text = year_placeholder(text)
    text = month_placeholder(text)
    text = person_name_placeholder(data, text)
    text = genre_placeholder(text)
    return text

In [18]:
print(get_paragraph_placeholder('data/1599.json'))

['[BAND_NAME] (Spanish for Wizard of Oz, with', 'metal umlaut) are', 'Spanish folk metal band from Begoña, Madrid formed in mid-[YEAR] by drummer Txus di Fellatio.\nThe band became well known for the strong Celtic feel to their music strengthened through their consistent usage of', 'violinist and flautist. The name for the band was chosen, according to founding member Txus, because "life is', 'yellow brick road, on which we walk in the company of others searching for our dreams." On the 26th of [MONTH], [YEAR], the band played', 'special concert to celebrate their 30th anniversary, playing with', 'symphony orchestra at the WiZink Center in Madrid. \nThe band was founded in [YEAR] by the drummer Txus di Fellatio, who slowly recruited the rest of the members for the band, finalising the line-up by [YEAR]. Originally, the band was called "Transilvania" ("Transylvania" in Spanish), named after', 'instrumental piece by Iron Maiden,', 'heavy influence on the band. In [YEAR], the band members

error: unbalanced parenthesis at position 12

In [12]:
for i in range(1657):
    try:
        get_paragraph_placeholder('data/' + str(i) + '.json')
    except:
        print(i)

758
1599
1648
